### **Importing Libraries**

In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from networkx.algorithms.community.centrality import girvan_newman
from operator import mul
import networkx as nx
import os
import sys
import community.community_louvain as community
import matplotlib.image as mpimg
import matplotlib.colors as mpcolors
import matplotlib.cm as mpcm
import random

### **Loading DataSet**

In [3]:
data_url = "./Dataset/Cit-HepPh.txt"
df_data_1 = pd.read_csv(data_url, sep='\t', skiprows=4, names=['FromNodeId', 'ToNodeId'], dtype={'FromNodeId': int, 'ToNodeId': int})

### **Loading Time of Release**

In [4]:
data_url = "./Dataset/cit-HepPh-dates.txt"
df_data_2 = pd.read_csv(data_url, sep='\t', skiprows=1, names=['NodeId', 'Date'], dtype={'NodeId': str, 'Date': str})
df_data_2['Date'] = pd.to_datetime(df_data_2['Date'])
df_data_2 = df_data_2[~df_data_2['NodeId'].str.startswith('11')]
df_data_2['NodeId'] = df_data_2['NodeId'].astype(str).str.lstrip('0')
df_data_2['NodeId'] = df_data_2['NodeId'].astype(int)
df_data_2 = df_data_2[df_data_2['Date'].dt.year <= 1992]
# i = 0
# unnodes = df_data_2['NodeId']
# for nodes in unnodes:
#     i += 1
# print(i)


### **Merging Both DataSet**

In [5]:
df_merged = pd.merge(df_data_1, df_data_2, how='inner', left_on='FromNodeId', right_on='NodeId')
df_merged['Date'] = pd.to_datetime(df_merged['Date'])
# Filter out rows where 'ToNodeId' is not present in 'NodeId' column of df_data_2
df_merged = df_merged[df_merged['ToNodeId'].isin(df_data_2['NodeId'])]
unnodes = df_merged['FromNodeId'].unique()
# i = 0
# for nodes in unnodes:
#     i += 1
# print(i)

### **Creation of Graph**

In [6]:

# Construct the directed graph
G_lat = nx.from_pandas_edgelist(df_merged, 'FromNodeId', 'ToNodeId', create_using=nx.DiGraph())

print("Number of nodes:", len(G_lat.nodes()))
print("Number of edges:", len(G_lat.edges()))
print(nx.density(G_lat))



Number of nodes: 173
Number of edges: 152
0.005108213469552359


### **Girvan-Newman Algorithm**

### **Method 1**

In [7]:
def edge_to_remove(g):
    d1 = nx.edge_betweenness_centrality(g) 
    list_of_tuples = list(d1.items()) 
      
    sorted(list_of_tuples, key = lambda x:x[1], reverse = True) 
      
    # Will return in the form (a,b) 
    return list_of_tuples[0][0] 

def girvan(graph):
    graph_cp = graph.copy()

    init_comp = nx.number_weakly_connected_components(graph_cp)
    while graph_cp.number_of_edges() > 0:

        u ,v = edge_to_remove(graph_cp)
        graph_cp.remove_edge(u,v)

        new_comp = nx.number_weakly_connected_components(graph_cp)

        # if new_comp > init_comp:
        #     break;
        if not nx.is_weakly_connected(graph_cp):
            break
    return list(nx.weakly_connected_components(graph_cp))

scratch_communities = girvan(G_lat)

# print("Final communities:", communities)
# for community in scratch_communities:
#     print(community)

### **Method 2**

In [8]:
class community_algo:
    def __init__(self, graph):
        self.graph = graph
        
    """"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
    """                               Main Functions                                 """
    """                                                                              """    
    """"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
    def calculate_directed_modularity(self, G, partition):
        m = G.number_of_edges()
        modularity = 0.0
        for community in set(partition.values()):
            nodes_in_community = [node for node, comm in partition.items() if comm == community]
            subgraph = G.subgraph(nodes_in_community)
            e_wc = sum([subgraph.out_degree(node, weight='weight') for node in nodes_in_community])
            a_c_out = sum([G.out_degree(node, weight='weight') for node in nodes_in_community])
            a_c_in = sum([G.in_degree(node, weight='weight') for node in nodes_in_community])
            modularity += (e_wc / m) - ((a_c_out / (2 * m)) * (a_c_in / (2 * m)))
        return modularity
    
    def find_best_partition(self):
        G = self.graph.copy()
        g_modularity = 0.0
        removed_edges = []
        partition = {}
        while 1:
            betweenness = self.calculate_betweenness(G)
            max_betweenness_edges = self.get_max_betweenness_edges(betweenness)
            if len(G.edges()) == len(max_betweenness_edges):
                break

            G.remove_edges_from(max_betweenness_edges)  
            components = nx.weakly_connected_components(G)
            idx = 0
            tmp_partition = {}
            for component in components:
                for inner in list(component):
                    tmp_partition.setdefault(inner, idx)
                idx += 1
            cur_mod = self.calculate_directed_modularity(G, tmp_partition)

            if cur_mod < g_modularity:
                G.add_edges_from(max_betweenness_edges)
                break
            else:
                partition = tmp_partition
            removed_edges.extend(max_betweenness_edges)
            g_modularity = cur_mod
        return partition, G, removed_edges

    def get_max_betweenness_edges(self, betweenness):
        max_betweenness_edges = []
        max_betweenness = max(betweenness.items(), key=lambda x: x[1])
        for (k, v) in betweenness.items():
            if v == max_betweenness[1]:
                max_betweenness_edges.append(k)
        return max_betweenness_edges

    def calculate_betweenness(self, G):
        betweenness = nx.edge_betweenness_centrality(G, k=None, normalized=True, weight=None, seed=None)
        return betweenness
    
    def partition_to_community(self, partition):
        result = {}
        for (k, v) in partition.items():
            result.setdefault(v, [])
            result[v].append(k)
        return result.values()

    def display(self, partition):
        comms = self.partition_to_community(partition)
        for comm in comms:
            comm.sort() # actually duplicate process here
            print(comm)
    
c = community_algo(G_lat)
partition, part_graph, removed_edges = c.find_best_partition()

inbuilt_communities = c.partition_to_community(partition)


### **Method 3**

In [11]:
# Use the inbuilt Girvan-Newman algorithm to detect communities
inbuilt_communities_generator = girvan_newman(G_lat,most_valuable_edge=None)
inbuilt_communities = [c for c in next(inbuilt_communities_generator)]

# for community in inbuilt_communities:
#     print(community)


# Assign random colors to each detected community ensuring uniqueness
# community_colors = {}
# used_colors = set()  # Set to store used colors
# for i, community in enumerate(scratch_communities):
#     color = "#{:06x}".format(random.randint(0, 0xFFFFFF))  # Generate a random hex color
#     community_colors[i] = color
#     used_colors.add(color)
#     for node in community:
#         G_lat.nodes[node]['community'] = i
#         G_lat.nodes[node]['color'] = color

# # Export the graph to GraphML format
# nx.write_graphml(G_lat, "graph_with_inbuilt_communities_unique_colors.graphml")
